In [1]:
import os
os.chdir('..')

In [2]:
from kernel.type import BoolType
from kernel.term import Var, And, Or, Implies, Eq, Inst, Not
from kernel.proofterm import ProofTerm, refl
from kernel import term_ord
from kernel import theory
from logic import basic
from logic.conv import Conv
from logic.logic import apply_theorem
from syntax.settings import settings

basic.load_theory('logic')
settings.unicode = True

## Resolution

The resolution rule (for propositional logic) is the following:

$$ \frac{A_1\vee\dots A_{i-1}\vee P\vee A_{i+1}\vee \dots \vee A_m \quad 
B_1\vee\dots B_{j-1}\vee \neg P\vee B_{j+1}\vee \dots\vee B_n}
{A_1\vee\dots A_{i-1}\vee A_{i+1}\vee\dots A_m \vee B_1\vee\dots B_{j-1}\vee B_{j+1}\vee\dots B_n} $$

That is, given two disjunctive clauses, one containing $P$ and one containing $\neg P$, we can *resolve* on $P$, removing it from both clauses and combining the clauses together. Resolution (with its extension to predicate logic) is the fundamental rule for a family of automatic theorem proving methods called resolution theorem proving. In addition, DPLL-based methods for propositional SAT solving can also produce independently checkable proofs as a list of resolution steps.

In this section, we describe implementation of resolution in higher-order logic. The methods used in this section comes from the paper "Fast LCF-Style Proof Reconstruction for Z3" by Sascha Bohme and Tjark Weber.

## Implication between conjunctions

First, we do a small exercise which will also be useful in the future. We wish to automatically prove statements of the form $A_1\wedge \dots \wedge A_m \to B_1\wedge\dots\wedge B_n$, where $\{B_1,\dots,B_n\}$ is a subset of $\{A_1,\dots,A_m\}$. This includes permutative problems like

$$ P \wedge Q \wedge R \to Q \wedge R \wedge P, $$

but also other situations involving missing or repeated conjuncts:

$$ R \wedge P \wedge Q \wedge R \to P \wedge P \wedge R. $$

For solving problems like the first example, one idea that immediately comes to mind is to apply the approach of the previous section, using commutativity and associativity to reorder the conjuncts. However, extending this approach to cover problems like the second example is more involved. Moreover, this approach is not very efficient, requiring $O(n^2)$ rewrites in general. We shall see soon that another approach is both simpler and more efficient.

The idea is as follows. First, we construct proofs for each conjunct in the assumption from the assumption, using the theorems `conjD1` and `conjD2`. This gives also gives us proofs for each conjunct in the conclusion due to the subset condition. Then, we use `conjI` to combine the proofs together to give the conclusion. The implementation is as follows:

In [3]:
def imp_conj(goal):
    """Goal is of the form A_1 & ... & A_m --> B_1 & ... & B_n, where
    {B_1, ..., B_n} is a subset of {A_1, ..., A_m}."""
    
    # Dictionary from conjuncts in A to its proof
    pts_A = dict()

    # Fills up pts_A from the assumption.
    def traverse_A(pt):
        if pt.prop.is_conj():
            traverse_A(apply_theorem('conjD1', pt))
            traverse_A(apply_theorem('conjD2', pt))
        else:
            pts_A[pt.prop] = pt
            
    # Use pts_A to prove the conclusion
    def traverse_C(t):
        if t.is_conj():
            left_pt = traverse_C(t.arg1)
            right_pt = traverse_C(t.arg)
            return apply_theorem('conjI', left_pt, right_pt)
        else:
            assert t in pts_A, "imp_conj: %s not found in assumption" % t
            return pts_A[t]

    A = goal.arg1
    traverse_A(ProofTerm.assume(A))
    return traverse_C(goal.arg).implies_intr(A)

Let us test this function on some examples:

In [4]:
P = Var('P', BoolType)
Q = Var('Q', BoolType)
R = Var('R', BoolType)

print(imp_conj(Implies(And(P, Q, R), And(R, Q, P))))
print(imp_conj(Implies(And(R, P, Q, R), And(P, P, R))))

ProofTerm(⊢ P ∧ Q ∧ R ⟶ R ∧ Q ∧ P)
ProofTerm(⊢ R ∧ P ∧ Q ∧ R ⟶ P ∧ P ∧ R)


As expected, the function raises an exception if the conjuncts in the conclusion is not a subset of conjuncts in the assumption:

In [5]:
imp_conj(Implies(And(P, Q), And(P, R)))  # raises AssertionError

AssertionError: imp_conj: R not found in assumption

To show an if-and-only-if relation between two conjuncts, we just compose the two directions using `equal_elim`:

In [6]:
def imp_conj_iff(goal):
    """Goal is of the form A_1 & ... & A_m <--> B_1 & ... & B_n, where
    the sets {A_1, ..., A_m} and {B_1, ..., B_n} are equal."""
    pt1 = imp_conj(Implies(goal.lhs, goal.rhs))
    pt2 = imp_conj(Implies(goal.rhs, goal.lhs))
    return ProofTerm.equal_intr(pt1, pt2)

In [7]:
print(imp_conj_iff(Eq(And(P, Q, R), And(R, Q, P))))
print(imp_conj_iff(Eq(And(R, P, Q, R), And(P, P, R, Q))))

ProofTerm(⊢ P ∧ Q ∧ R ⟷ R ∧ Q ∧ P)
ProofTerm(⊢ R ∧ P ∧ Q ∧ R ⟷ P ∧ P ∧ R ∧ Q)


Based on this, we can also implement a normalization conversion. The conversion takes a conjunction and sort the conjuncts (as a Python list), then creates the desired equality for `imp_conj_iff` to prove:

In [8]:
def strip_conj(t):
    def rec(t):
        if t.is_conj():
            return rec(t.arg1) + rec(t.arg)
        else:
            return [t]
    return term_ord.sorted_terms(rec(t))

class conj_norm(Conv):
    """Normalize an conjunction."""
    def get_proof_term(self, t):
        goal = Eq(t, And(*strip_conj(t)))
        return imp_conj_iff(goal)

In [9]:
def test_conv(cv, ts):
    for t in ts:
        print(refl(t).on_rhs(cv).prop)

test_conv(conj_norm(), [
    And(Q, R, P),
    And(P, P, Q, Q, R, R),
])

Q ∧ R ∧ P ⟷ P ∧ Q ∧ R
P ∧ P ∧ Q ∧ Q ∧ R ∧ R ⟷ P ∧ Q ∧ R


## Implication between disjunctions

The same idea can be applied to showing implication between disjunctions. The algorithm is just slightly more complicated. Here, we need to show $A_1\vee\dots\vee A_m \to B_1\vee\dots\vee B_n$, where $\{A_1,\dots,A_m\}$ is a subset of $\{B_1,\dots,B_n\}$. The idea is to show $B_i\to B_1\vee\dots\vee B_n$ for each $i$, using theorems `disjI1` and `disjI2`, then combine the results together using `disjE`. The implementation is as follows:

In [10]:
def imp_disj(goal):
    """Goal is of the form A_1 | ... | A_m --> B_1 | ...| B_n, where
    {A_1, ..., A_m} is a subset of {B_1, ..., B_n}."""

    # Dictionary from B_i to B_i --> B_1 | ... | B_n
    pts_B = dict()
    
    # Fills up pts_B.
    def traverse_C(pt):
        if pt.prop.arg1.is_disj():
            pt1 = apply_theorem('disjI1', concl=pt.prop.arg1)
            pt2 = apply_theorem('disjI2', concl=pt.prop.arg1)
            traverse_C(apply_theorem('syllogism', pt1, pt))
            traverse_C(apply_theorem('syllogism', pt2, pt))
        else:
            pts_B[pt.prop.arg1] = pt
    
    # Use pts_B to prove the implication
    def traverse_A(t):
        if t.is_disj():
            pt1 = traverse_A(t.arg1)
            pt2 = traverse_A(t.arg)
            return apply_theorem('disjE2', pt1, pt2)
        else:
            assert t in pts_B, "imp_disj: %s not found in conclusion" % t
            return pts_B[t]
        
    triv = apply_theorem('trivial', inst=Inst(A=goal.arg))
    traverse_C(triv)
    return traverse_A(goal.arg1)

We test this on some examples:

In [11]:
P = Var('P', BoolType)
Q = Var('Q', BoolType)
R = Var('R', BoolType)

print(imp_disj(Implies(Or(P, Q, R), Or(R, Q, P))))
print(imp_disj(Implies(Or(P, P, R), Or(R, P, Q, R))))

ProofTerm(⊢ P ∨ Q ∨ R ⟶ R ∨ Q ∨ P)
ProofTerm(⊢ P ∨ P ∨ R ⟶ R ∨ P ∨ Q ∨ R)


As above, we can implement a function for showing if-and-only-if statements:

In [12]:
def imp_disj_iff(goal):
    """Goal is of the form A_1 | ... | A_m <--> B_1 | ... | B_n, where
    the sets {A_1, ..., A_m} and {B_1, ..., B_n} are equal."""
    pt1 = imp_disj(Implies(goal.lhs, goal.rhs))
    pt2 = imp_disj(Implies(goal.rhs, goal.lhs))
    return ProofTerm.equal_intr(pt1, pt2)

In [13]:
print(imp_disj_iff(Eq(Or(P, Q, R), Or(R, Q, P))))
print(imp_disj_iff(Eq(Or(R, P, Q, R), Or(P, P, R, Q))))

ProofTerm(⊢ P ∨ Q ∨ R ⟷ R ∨ Q ∨ P)
ProofTerm(⊢ R ∨ P ∨ Q ∨ R ⟷ P ∨ P ∨ R ∨ Q)


Finally, we define a conversion for normalizing a disjunction.

In [14]:
def strip_disj(t):
    def rec(t):
        if t.is_disj():
            return rec(t.arg1) + rec(t.arg)
        else:
            return [t]
    return term_ord.sorted_terms(rec(t))

class disj_norm(Conv):
    """Normalize an disjunction."""
    def get_proof_term(self, t):
        goal = Eq(t, Or(*strip_disj(t)))
        return imp_disj_iff(goal)

In [15]:
test_conv(disj_norm(), [
    Or(Q, R, P),
    Or(P, P, Q, Q, R, R),
])

Q ∨ R ∨ P ⟷ P ∨ Q ∨ R
P ∨ P ∨ Q ∨ Q ∨ R ∨ R ⟷ P ∨ Q ∨ R


## Proof construction for resolution

We now consider the problem of constructing proof for a resolution step. The basic procedure is as follows. First, we traverse the two disjunction to find a literal to resolve on. Then, this literal is moved to the front. This allows the resolution theorem to be applied, and we normalize the results. There are corner cases where the left or right side contains only one disjunct, and we use separate theorems for these:

In [16]:
theory.print_theorem('resolution', 'resolution_left', 'resolution_right', 'negE')

resolution: ⊢ A ∨ B ⟶ ¬A ∨ C ⟶ B ∨ C
resolution_left: ⊢ A ∨ B ⟶ ¬A ⟶ B
resolution_right: ⊢ A ⟶ ¬A ∨ B ⟶ B
negE: ⊢ ¬A ⟶ A ⟶ false


In [17]:
def resolution(pt1, pt2):
    """Input proof terms are A_1 | ... | A_m and B_1 | ... | B_n, where
    there is some i, j such that B_j = ~A_i or A_i = ~B_j."""
    
    # First, find the pair i, j such that B_j = ~A_i or A_i = ~B_j, the
    # variable side records the side of the positive literal.
    disj1 = strip_disj(pt1.prop)
    disj2 = strip_disj(pt2.prop)
    
    side = None
    for i, t1 in enumerate(disj1):
        for j, t2 in enumerate(disj2):
            if t2 == Not(t1):
                side = 'left'
                break
            elif t1 == Not(t2):
                side = 'right'
                break
        if side is not None:
            break
            
    assert side is not None, "resolution: literal not found"
    
    # If side is wrong, just swap:
    if side == 'right':
        return resolution(pt2, pt1)
    
    # Move items i and j to the front
    disj1 = [disj1[i]] + disj1[:i] + disj1[i+1:]
    disj2 = [disj2[j]] + disj2[:j] + disj2[j+1:]
    eq_pt1 = imp_disj_iff(Eq(pt1.prop, Or(*disj1)))
    eq_pt2 = imp_disj_iff(Eq(pt2.prop, Or(*disj2)))
    pt1 = eq_pt1.equal_elim(pt1)
    pt2 = eq_pt2.equal_elim(pt2)
    
    if len(disj1) > 1 and len(disj2) > 1:
        pt = apply_theorem('resolution', pt1, pt2)
    elif len(disj1) > 1 and len(disj2) == 1:
        pt = apply_theorem('resolution_left', pt1, pt2)
    elif len(disj1) == 1 and len(disj2) > 1:
        pt = apply_theorem('resolution_right', pt1, pt2)
    else:
        pt = apply_theorem('negE', pt2, pt1)

    return pt.on_prop(disj_norm())

We test this on some examples:

In [18]:
test_data = [
    (Or(P, Q), Or(Not(P), R)),
    (Or(P, Q, R), Or(P, Not(Q), R)),
    (Or(P, Q), Not(P)),
    (Not(Q), Or(P, Q)),
    (P, Not(P)),
    (Not(P), P),
]

for t1, t2 in test_data:
    print(resolution(ProofTerm.assume(t1), ProofTerm.assume(t2)))

ProofTerm(P ∨ Q, ¬P ∨ R ⊢ Q ∨ R)
ProofTerm(P ∨ Q ∨ R, P ∨ ¬Q ∨ R ⊢ P ∨ R)
ProofTerm(¬P, P ∨ Q ⊢ Q)
ProofTerm(¬Q, P ∨ Q ⊢ P)
ProofTerm(P, ¬P ⊢ false)
ProofTerm(P, ¬P ⊢ false)
